# Лабораторная работа 4 

### Импорт зависимостей

In [14]:
import pykeen
import torch
from pykeen.datasets import PharmKG8k
from pykeen.pipeline import pipeline
from pykeen.predict import predict_triples

### Загрузка датасета

In [5]:
dataset = PharmKG8k()

In [6]:
print(dataset)

PharmKG8k(training_path="C:\Users\goras\.data\pykeen\datasets\pharmkg8k\train.tsv", testing_path="C:\Users\goras\.data\pykeen\datasets\pharmkg8k\test.tsv", validation_path="C:\Users\goras\.data\pykeen\datasets\pharmkg8k\valid.tsv")


In [7]:
training_data = dataset.training.mapped_triples
print(training_data[:5])

You're trying to map triples with 7 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 7 from 50036 triples were filtered out


tensor([[   0,   13, 6562],
        [   2,   13, 4427],
        [   2,   13, 6038],
        [   2,   13, 6986],
        [   2,   26,  330]])


### Инициализация и обучение модели

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [12]:
pipeline_results = pipeline(
    model="TransE",      # Выбираем модель (TransE - одна из моделей для предсказания отношений)
    dataset=dataset,     # Выбираем графовый датасет (Nations)
    training_kwargs=dict(num_epochs=20),   # Количество эпох обучения
    device=device
)

No random seed is specified. Setting to 3097829094.
You're trying to map triples with 12 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 12 from 49536 triples were filtered out


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/1511 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/1511 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/1511 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/1511 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/1511 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
C:\Users\goras\miniconda3\envs\AI_testing\Lib\site-packages\pykeen\utils.py:545: RuntimeWarning: overflow encountered in scalar add
  return sum(
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=4096.


Evaluating on cuda:0:   0%|          | 0.00/49.8k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 14.15s seconds


In [13]:
print(pipeline_results)

PipelineResult(random_seed=3097829094, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(7247, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(28, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=7247, num_relations=28, create_inverse_triples=False, num_triples=386768, path="C:\Users\goras\.data\pykeen\datasets\pharmkg8k\train.tsv"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x0000021F2DA8FB50>, losses=[0.9050896000893679, 0.4821024591326319, 0.30059280515229914, 0.22956995663068533, 0.19556266196136204, 0.17271366865330387, 0.15925558402019965, 0.1458574859320644, 0.13915810684231555, 0.1307797398760885, 0.12624521926538507, 0.1220932830122628, 0.11928821837345864, 0.11655168867442563, 0.11368695187458212, 0

### Triple Scoring

In [15]:
pack = predict_triples(model=pipeline_results.model, triples=dataset.validation)
df = pack.process(factory=pipeline_results.training).df

In [17]:
df.nlargest(n=100, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
161,19,abcb11,20,Ra,19,abcb11,-2.153477
1484,219,ahsg,20,Ra,219,ahsg,-2.153477
2781,442,apoe,20,Ra,442,apoe,-2.153477
2971,461,ar,20,Ra,461,ar,-2.153477
4344,696,bcor,20,Ra,696,bcor,-2.153477
...,...,...,...,...,...,...,...
9295,1344,cish,21,Rg,1344,cish,-2.284354
9471,1360,clcn2,21,Rg,1360,clcn2,-2.284354
9492,1367,cldn1,21,Rg,1367,cldn1,-2.284354
9501,1370,cldn18,21,Rg,1370,cldn18,-2.284354
